In [1]:
pip install mysql-connector-python

In [5]:
import mysql.connector
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from mysql.connector import Error

id=0
html = urlopen('https://one-versus-one.com/en/rankings')
soup = BeautifulSoup(html, 'html.parser')

base_url = "https://one-versus-one.com/en/rankings"

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# Function to execute SQL queries
def execute_query(connection, query,data=None):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

# Create MySQL connection
connection = create_connection("localhost", "", "", "train")

create_players_table = """
CREATE TABLE IF NOT EXISTS players (
  id INT AUTO_INCREMENT PRIMARY KEY,
  player_name VARCHAR(255),
  team VARCHAR(255),
  rating FLOAT
)
"""

execute_query(connection, create_players_table)

create_player_info_table = """
CREATE TABLE IF NOT EXISTS PlayerInfo (
  id INT AUTO_INCREMENT PRIMARY KEY,
  Name VARCHAR(255),
  Club VARCHAR(255),
  Nationality VARCHAR(255),
  Position VARCHAR(255),
  Age VARCHAR(255),
  Height VARCHAR(255),
  preferred_foot VARCHAR(255)
)
"""
execute_query(connection, create_player_info_table)


create_transfers_table = """
CREATE TABLE IF NOT EXISTS transfers (
  id INT AUTO_INCREMENT PRIMARY KEY,
  Name VARCHAR(255),
  season VARCHAR(255),
  club VARCHAR(255),
  date_of_transfer VARCHAR(255),
  duration VARCHAR(255)
)
"""
execute_query(connection, create_transfers_table)


create_trophies_table = """
CREATE TABLE IF NOT EXISTS trophies (
  id INT AUTO_INCREMENT PRIMARY KEY,
  Name VARCHAR(255),
  trophy VARCHAR(255),
  amount VARCHAR(255),
  year VARCHAR(255)
)
"""
execute_query(connection, create_trophies_table)

# Create metrics table
create_metrics_table = """
CREATE TABLE IF NOT EXISTS metrics (
  id INT AUTO_INCREMENT PRIMARY KEY,
  Name VARCHAR(255),
  WeeklyTotalPayout VARCHAR(255),
  YearlyTotalPayout VARCHAR(255),
  MatchesPlayed VARCHAR(255),
  MinutesPlayed VARCHAR(255),
  Goals VARCHAR(255),
  GoalsPer90min VARCHAR(255),
  Assists VARCHAR(255),
  AssistsPer90min VARCHAR(255),
  Progressivedribbles VARCHAR(255),
  Successfulldribbles VARCHAR(255),
  SuccessfulldribblesPer90min VARCHAR(255)
)
"""

execute_query(connection, create_metrics_table)


for page_num in range(1, 341): 
    url = base_url + "?page=" + str(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    a_tags = soup.find_all('a', class_='ranking-table__item-content')
    span_elements = soup.find_all('span', class_='ranking-table__item-name')
    img_elements = soup.find_all('img', class_='ranking-table__item-team-image')
    rating_elements = soup.find_all('span', class_='ranking-table__item-rating mobile-h1')
    

    for index in range(len(span_elements)):
        link = a_tags[index]['href']
        url2 = link
        response = requests.get(url2)
        html = response.text
        soup = BeautifulSoup(html,'html.parser')
        player_name_element = soup.find('span', class_='text-white leading-none h1-48')
        try:
            player_name = player_name_element.text.strip()

            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            team = img_elements[index]['alt']
            rating = float(rating_elements[index].text.strip())

            insert_query = f"""
            INSERT INTO players (player_name, team, rating) 
            VALUES ('{player_name}', '{team}', {rating})
            """
            execute_query(connection, insert_query)

            link = a_tags[index]['href']
            url2 = link
            response = requests.get(url2)
            html = response.text
            soup = BeautifulSoup(html,'html.parser')
            rows = soup.find_all('tr')

            Club = None
            Nationality = None
            Position = None
            Age = None
            Height = None
            preferred_foot = None

            for row in rows:
                th = row.find('th')
                if th:
                    label = th.get_text(strip=True)
                    if label == 'Club':
                        Club = row.find('td').get_text(strip=True)
                    elif label == 'Nationality':
                        Nationality = row.find('td').get_text(strip=True)
                    elif label == 'Position':
                        Position = row.find('td').get_text(strip=True)
                    elif label == 'Age':
                        Age = row.find('td').get_text(strip=True)
                    elif label == 'Height':
                        Height = row.find('td').get_text(strip=True)
                    elif label == 'Foot':
                        preferred_foot = row.find('td').get_text(strip=True)

            insert_query = f"""
            INSERT INTO PlayerInfo (Name,Club,Nationality,Position,Age, Height,preferred_foot) 
            VALUES ('{player_name}','{Club}', '{Nationality}', '{Position}','{Age}','{Height}','{preferred_foot}')
            """
            execute_query(connection, insert_query)

            season_elements = soup.find_all('td', class_='border-b border-gray-high body-min-bold text-gray-mid text-left p-0 pb-2 pt-4')
            club_elements = soup.find_all('span', class_='body-min-bold ml-2')
            date_elements = soup.select('div.body-small-bold')
            duration_elements = soup.find_all('span', class_='body-min-bold text-gray-mid')

            for season, club, date, duration in zip(season_elements, club_elements, date_elements, duration_elements):
                season_text = season.get_text(strip=True)
                club_text = club.get_text(strip=True)
                date_text = date.get_text(strip=True).replace('\n', '').replace("'", ' ')
                duration_text = duration.get_text(strip=True)      
                duration_text = duration_text.replace('YRS', 'Years').replace('yrs', 'Years').replace('yr','Years')
                duration_text = duration_text.replace('MOS', 'Months').replace('mos', 'Months')
                result = ''
                for i in range(len(duration_text)):
                    if duration_text[i].isdigit() and i < len(duration_text) - 1 and duration_text[i + 1].isalpha():
                        result += duration_text[i] + ' '
                    else:
                        result += duration_text[i]
                duration_text = result


                insert_query = f"""
                INSERT INTO transfers (Name,season,club,date_of_transfer,duration) 
                VALUES ('{player_name}','{season_text}', '{club_text}', '{date_text}','{duration_text}')
                """

                execute_query(connection, insert_query)



            trophy_items = soup.find_all('tr', class_='trophies-table__item')

            for item in trophy_items:
                trophy_div = item.find('td', class_='border-b border-gray-high text-left p-0 pb-2 pt-4').find('div', class_='body-min-bold whitespace-nowrap')
                trophy = trophy_div.get_text(strip=True)
                amount_div = item.find_all('td', class_='border-b border-gray-high text-left p-0 pb-2 pt-4')[1].find('div', class_='body-min-bold whitespace-nowrap')
                amount = amount_div.get_text(strip=True)
                year_div = item.find('td', class_='border-b border-gray-high text-right p-0 pb-2 pt-4 max-w-[116px]').find('div', class_='body-min-bold text-gray-mid text-right')
                year = year_div.get_text(strip=True)


                insert_query = f"""
                INSERT INTO trophies (Name,trophy,amount,year) 
                VALUES ('{player_name}','{trophy}', '{amount}', '{year}')
                """

                execute_query(connection, insert_query)

            rows = soup.find_all('tr')
            array = []
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    metric = cells[0].find('span').get_text(strip=True)
                    value = cells[1].get_text(strip=True)
                    array.append(value)


            insert_query = f"""
            INSERT INTO metrics (Name,WeeklyTotalPayout,YearlyTotalPayout,MatchesPlayed,MinutesPlayed,Goals,GoalsPer90min,
            Assists,AssistsPer90min,Progressivedribbles,Successfulldribbles,SuccessfulldribblesPer90min) 
            VALUES ('{player_name}','{array[0]}', '{array[1]}', '{array[2]}', '{array[3]}', '{array[4]}', '{array[5]}', '{array[6]}'
            , '{array[7]}', '{array[8]}', '{array[9]}', '{array[10]}')
            """

            execute_query(connection, insert_query)     
        except Exception as e:
            print("An unexpected error occurred:", e)
    
if connection:
    connection.close()


Connection to MySQL DB successful
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

...

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute

An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute

An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute 'text'
An unexpected error occurred: 'NoneType' object has no attribute